## Geo-Locations zu zusätzlichen Merkmalen transformieren

In [1]:
# ! pip install -U pandas geopandas geopy folium scikit-learn

Wir arbeiten wieder mit Pandas, aber auch der Erweiterung GeoPandas, welche die Arbeit mit Koordinaten-basierten Daten vereinfacht. Zudem benötigen wir wieder SciKit-Learn, um ein ML-Modell zu trainieren und zu evaluieren.

In [2]:
import pandas as pd
import geopandas as gpd
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split

In diesem Fall haben wir einen vorliegenden Datensatz als GeoJSON, welcher Koordinaten und einen Zielwert zur Vorhersage enthält.

In [3]:
data = gpd.read_file("../02_resources/dataset.geo.json")
data.explore()

In [4]:
data

,y,geometry
0,764.609049,POINT (11.63259 48.21601)
1,1173.510463,POINT (11.60593 48.17391)
2,507.098209,POINT (11.44986 48.15745)
3,503.555922,POINT (11.56693 48.17549)
4,772.215181,POINT (11.67001 48.12629)
...,...,...
245,682.087469,POINT (11.52879 48.17093)
246,519.902908,POINT (11.54309 48.21340)
247,713.903267,POINT (11.70174 48.14237)
248,1522.098992,POINT (11.39557 48.13642)


Wenn wir nun ein ML-Modell mit sklearn trainieren wollen, müssten wir auf die X und Y Koordinaten zurückgreifen

In [5]:
X = pd.DataFrame([data.geometry.y, data.geometry.x]).transpose() # Merkmale
y = data.y # Zielwert

X

,0,1
0,48.216010,11.632594
1,48.173908,11.605926
2,48.157446,11.449862
3,48.175491,11.566931
4,48.126290,11.670015
...,...,...
245,48.170928,11.528787
246,48.213399,11.543091
247,48.142374,11.701736
248,48.136420,11.395569


In [6]:
reg = GradientBoostingRegressor()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

reg = reg.fit(X_train, y_train)
reg.score(X_test, y_test)

0.7871586391319553

Die Vorhersage anhand der Koordinaten läuft also nicht zu gut. Wir können versuchen, stattdessen die Distanz zur nächsten Haltestelle des MVV als Merkmal zu nutzen. 

Diese gibt es im Datensatz auf dem Open Data Portal: https://opendata.muenchen.de/dataset/haltestellenliste-mvv

In [7]:
df = pd.read_csv("../02_resources/20-01-MVV_HST-_s20-o-T.csv", sep=";", decimal=",")
df = df[df.Ort == "München"]
df= df.dropna()

stations = gpd.GeoDataFrame(data=df["Name ohne Ort"], geometry=gpd.points_from_xy(df["WGS84 Y"], df["WGS84 X"])).set_crs(epsg=4326)
stations

,Name ohne Ort,geometry
0,Karlsplatz (Stachus),POINT (11.56539 48.13937)
1,Marienplatz,POINT (11.57765 48.13643)
2,Isartor,POINT (11.58282 48.13389)
3,Rosenheimer Platz,POINT (11.59389 48.12859)
4,Ostbahnhof,POINT (11.60406 48.12814)
...,...,...
5673,Moosacher Str. 51,POINT (11.55734 48.18554)
5674,Lauchstädter Straße,POINT (11.57390 48.18903)
5679,Taunusstraße,POINT (11.57835 48.18891)
5680,BTZ Lemgostraße,POINT (11.55454 48.19214)


In [8]:
stations.explore()

Nun erstellen wir eine Funktion, die die Distanz zur nächsten Station (Luftlinie) berechnet und wenden diese auf unseren Datensatz an:

In [9]:
def get_nearest_station(point):
    return round(stations.to_crs(epsg=25832).distance(point).min())

data["nearest_station"] = data.geometry.to_crs(epsg=25832).apply(get_nearest_station)

data

,y,geometry,nearest_station
0,764.609049,POINT (11.63259 48.21601),270
1,1173.510463,POINT (11.60593 48.17391),636
2,507.098209,POINT (11.44986 48.15745),25
3,503.555922,POINT (11.56693 48.17549),28
4,772.215181,POINT (11.67001 48.12629),294
...,...,...,...
245,682.087469,POINT (11.52879 48.17093),136
246,519.902908,POINT (11.54309 48.21340),47
247,713.903267,POINT (11.70174 48.14237),175
248,1522.098992,POINT (11.39557 48.13642),1028


Wenn wir den Regressor nun mit dem neuen Merkmal nearest_station trainieren, erhalten wir einen weitaus besseren Score:

In [10]:
X, y = data.nearest_station.array.reshape(-1, 1), data.y
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

reg = reg.fit(X_train, y_train)
reg.score(X_test, y_test)

0.9960562547467282